In [2]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

#pd.set_option('display.max_rows', 4000)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db1/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
tart_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-06-30')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"

## notice that 43 doesn't exists in the db
#nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
#nodes=['01'] # test


In [4]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

jobs_to_nodes_whole_data contains 469095 records
There are 336787 long jobs in the Alina's interval


In [5]:
jobs_not_zero_quality = jobs[jobs['real_pow'] > 0]

jobs_test = jobs_not_zero_quality[pd.to_datetime(jobs_not_zero_quality['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) > train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))

test set contains 24918 records


In [6]:
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

after the clean, jobs_to_nodes_whole_data contains 444610 records


In [7]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_test, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 25569 records


In [8]:
nodes = pd.read_csv(infile_nodes, index_col=0)
#print(nodes.shape[0])
#nodes.head()

In [10]:
## create the file where to calculate the approximated consumption
#### part 1
#infile_node = datadir + "CPUs/" + interval_comment + "/node01" + suffix + interval_comment + "_active_cores_and_jobs"
#node_data = pd.read_csv(infile_node + ".csv")

#all_time_interval_power_consumption = pd.DataFrame(node_data['timestamp'])
#infile_validate_approx = datadir + "CPUs/" + interval_comment + "/validate_approx_perUserJob" + interval_comment + ".csv"
#all_time_interval_power_consumption.to_csv(infile_validate_approx)


### part 2
#all_time_interval_power_consumption = pd.read_csv(infile_validate_approx, index_col=0)
#all_time_interval_power_consumption.shape

In [33]:
import datetime

period = (test_end_time - train_end_time)
period = period.total_seconds() / 5 # total seconds / 5

interval_serie = pd.date_range(train_end_time, periods=period, freq='5s')
frame = { 'timestamp': interval_serie }

all_time_interval_power_consumption = pd.DataFrame(frame)

In [34]:
all_time_interval_power_consumption

,timestamp
0,2014-10-01 00:00:00
1,2014-10-01 00:00:05
2,2014-10-01 00:00:10
3,2014-10-01 00:00:15
4,2014-10-01 00:00:20
5,2014-10-01 00:00:25
6,2014-10-01 00:00:30
7,2014-10-01 00:00:35
8,2014-10-01 00:00:40
9,2014-10-01 00:00:45


In [67]:
def calculate_power_consumption(node_str, node_core_idle, interval_size, node_start_time):
    
    # num_minutes = 535680 
    # node_start_time = pd.to_datetime('2014-03-31 00:00:00')
    total_approx_power_consumption = pd.Series(0, index=np.arange(interval_size))
    total_active_cores = pd.Series(0, index=np.arange(interval_size))
    total_real_power_consumption = pd.Series(0, index=np.arange(interval_size))
    
    jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node_str)]
    #infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs"
    #node_data = pd.read_csv(infile_node + ".csv")
    
    power_consumption_real = pd.Series(0, index=np.arange(interval_size))
    power_consumption_pred = pd.Series(0, index=np.arange(interval_size))
    active_cores = pd.Series(0, index=np.arange(interval_size))
    
    i = 0
    num_jobs = jobs_to_node.shape[0]
    print("{}".format(num_jobs))
    for job_index, job_row in jobs_to_node.iterrows():
        
        #print("{}/{}".format(i, num_jobs))
        i = i + 1
        job_start_time = pd.to_datetime(job_row['run_start_time'])
        job_end_time = pd.to_datetime(job_row['end_time'])
        job_cores = job_row['ncpus']
        job_cores_total = job_row['cpu_req']
        #job_gpus = row['ngpus']
        #job_mics = row['nmics']
        job_power_consumption_pred = job_row['predict_online_whole'] * (job_cores / job_cores_total) # real_pow contains the total, here we need only the node portion
        job_power_consumption = job_row['real_pow'] * (job_cores / job_cores_total) # real_pow contains the total, here we need only the node portion
        
        
        # print(job_power_consumption_pred)

        before_minutes = int((job_start_time - pd.to_datetime(node_start_time) - np.timedelta64(5, 's')) / np.timedelta64(5, 's'))
        running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
        after_minutes = interval_size - running_minutes - before_minutes

        #pred
        before_serie = pd.Series(0, index=np.arange(before_minutes))
        running_serie = pd.Series(job_power_consumption_pred, index=np.arange(running_minutes))
        after_serie = pd.Series(0, index=np.arange(after_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        power_consumption_pred = power_consumption_pred.add(concat_series, fill_value=0)
        
        # real
        running_serie = pd.Series(job_power_consumption_real, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        power_consumption_real = power_consumption_real.add(concat_series, fill_value=0)

        running_serie = pd.Series(job_cores, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_cores = active_cores.add(concat_series, fill_value=0)

        
    for x in range(0, 17):
        # active_cores[active_cores == x] += (node_core_idle * 16) * (16 - x) / 16
        active_cores[active_cores == x] += node_core_idle * (16 - x)
    
    power_consumption_pred = power_consumption_pred.add(active_cores, fill_value=0)
    power_consumption_real = power_consumption_real.add(active_cores, fill_value=0)
        
    #indices = node_data['active_cores'] == 0
    #node_data.at[indices, 'pow_tot'] = node_core_idle * 16
    #total_real_power_consumption = node_data['pow_tot']
    
    return power_consumption_real, power_consumption_pred

In [68]:
infile_validate_approx = datadir + "CPUs/" + interval_comment + "/validate_pred_" + interval_comment + ".csv"

# per ogni nodo sommo quanto è stato il consumo ad ogni intervallo in un vettore parallelo
# questo sia per il valore approssimato guardando i job
# sia per il valore reale
j = 0
for node_index, node_row in nodes.iterrows():
    node_id = node_row['node_id']
    if( int(node_id) < 10):
        node_id = "0" + str(node_id)
    else:
        node_id = str(node_id)
    print(node_id)
    node_core_idle = node_row['core_idle']
    
    if (node_id != "43"):
        total_real_power_consumption, total_pred_power_consumption = calculate_power_consumption(node_id, node_core_idle)
        all_time_interval_power_consumption[node_id + "_real"] = total_real_power_consumption
        all_time_interval_power_consumption[node_id + "_pred"] = total_approx_power_consumption
    

    all_time_interval_power_consumption.to_csv(infile_validate_approx)

01
443
02
191
03
409
04
384
05
358
06
334
07
335
08
356
09
266
10
308
11
294
12
308
13
295
14
280
15
286
16
288
17
468
18
178
19
273
20
283
21
270
22
3
23
290
24
237
25
264
26
191
27
270
28
233
29
246
30
215
31
202
32
218
33
2358
34
1649
35
1030
36
1295
37
1081
38
942
39
101
40
1116
41
3
42
845
43
44
1909
45
1551
46
795
47
731
48
694
49
1490
50
973
51
905
52
598
53
13
54
689
55
1802
56
677
57
828
58
916
59
2828
60
1007
61
856
62
727
63
627
64
1204


In [23]:
#all_time_interval_power_consumption

In [23]:
all_time_interval_power_consumption = all_time_interval_power_consumption[pd.to_datetime(all_time_interval_power_consumption['timestamp']) >= pd.to_datetime('2014-03-31 11:21:00')]
all_time_interval_power_consumption = all_time_interval_power_consumption[pd.to_datetime(all_time_interval_power_consumption['timestamp']) <= pd.to_datetime('2014-04-30 20:32:00')]

In [24]:
#all_time_interval_power_consumption = all_time_interval_power_consumption.fillna(55.52)

In [42]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

def plot_line(data_dates, data_real, data_approx, days_interval):
    plt.rcParams['figure.figsize'] = (20,5)

    X_AXIS_NAME = 'Time (min)'
    Y_AXIS_NAME = 'Power (W)'
    # TITLE = 'Nodes power consumption - Andrea\'s data'

    fig, ax = plt.subplots()
    ax.xaxis.get_majorticklabels()
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=days_interval))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.tick_params(axis='both', which='major', labelsize=15)

    dates = []
    colors = ("orange", "blue")
    #colors = ("orange", "green")

    for ts in data_dates['timestamp']:
        local_d = datetime.strptime(ts, '%Y-%m-%d %H:%M:%S')
        dates.append(local_d)


    plt.plot(dates, data_real, c=colors[1], alpha=0.5)
    plt.plot(dates, data_real, c=colors[1])
    
    plt.plot(dates, data_approx, c=colors[0], alpha=1)

    #plt.plot(dates, data_real, c=colors[1], alpha=0.6, dashes=[2, 2])
    
    #plt.scatter(dates, node_data['pow_tot_0'] + node_data['pow_tot_1'])

    #fig.autofmt_xdate()
    plt.xlabel(X_AXIS_NAME, fontsize=20)
    plt.ylabel(Y_AXIS_NAME, fontsize=20)
    # plt.title(TITLE)

    # function to show the plot
    plt.show()
    #fig.savefig('graph.png', dpi=fig.dpi)

In [ ]:
infile_approximated = datadir + "CPUs/" + interval_comment + "/validate_approx_" + interval_comment + ".csv"
approximated_power_consumption = pd.read_csv(infile_approximated)

infile_predicted = datadir + "CPUs/" + interval_comment + "/validate_approx_perUserJob" + interval_comment + ".csv"
predicted_power_consumptions = pd.read_csv(infile_predicted)

approximated_power_consumption = approximated_power_consumption[pd.to_datetime(approximated_power_consumption['timestamp']) >= pd.to_datetime('2014-03-31 11:21:00')]
approximated_power_consumption = approximated_power_consumption[pd.to_datetime(approximated_power_consumption['timestamp']) <= pd.to_datetime('2014-04-30 20:32:00')]
predicted_power_consumptions = predicted_power_consumptions[pd.to_datetime(predicted_power_consumptions['timestamp']) >= pd.to_datetime('2014-03-31 11:21:00')]
predicted_power_consumptions = predicted_power_consumptions[pd.to_datetime(predicted_power_consumptions['timestamp']) <= pd.to_datetime('2014-04-30 20:32:00')]

approx = approximated_power_consumption.select(lambda col: col.endswith("approx"), axis=1)
real = predicted_power_consumptions.select(lambda col: col.endswith("real"), axis=1)
predicted = predicted_power_consumptions.select(lambda col: col.endswith("pred"), axis=1)

In [ ]:
approx['Total'] = approx.iloc[:, -64:-1].sum(axis=1)
real['Total'] = real.iloc[:, -64:-1].sum(axis=1)
predicted['Total'] = predicted.iloc[:, -64:-1].sum(axis=1)
differences = approx['Total'] - predicted['Total']
df = pd.DataFrame(differences)
df = df.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
differences_norm = df['Total']

#plot_hist(n_bins=150, data=differences)
#plot_hist(n_bins=150, data=differences_norm)

#plot_line(data_dates=approximated_power_consumption, data_real=real['Total'], data_approx=predicted['Total'], days_interval=5)
plot_line(data_dates=approximated_power_consumption, data_real=approx['Total'], data_approx=predicted['Total'], days_interval=5)

#print("approximation accuracy: {}".format(r2_score(real['Total'], approx['Total'])))